### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import pandas as pd
import sys

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data_complete_df
#test = school_data_complete_df({"size","budget"}).mean
#test

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
cnt = len(school_data["School ID"].unique())
tot_st = school_data["size"].sum()
tot_bd = school_data["budget"].sum()
district_summary_data = pd.DataFrame({
                                    "Total Schools": [cnt],
                                    "Total Students": tot_st,
                                    "Total Budget": tot_bd,
                                    "Rowid": 1
                                   })
# print(district_summary_data)
avg_read = student_data["reading_score"].mean()
avg_math = student_data["math_score"].mean()
# print(avg_read , avg_math)
# pass_mathcnt = len(student_data[student_data.math_score > 69])
pass_readcnt = len(student_data.loc[student_data["reading_score"] > 69, ["reading_score"]])
# pass_readcnt
pass_mathcnt = len(student_data.loc[student_data["math_score"] > 69, ["math_score"]])
# pass_mathcnt
# pass_rate = student_data.loc[((student_data["reading_score"] > 69) & (student_data["math_score"] > 69)), "Student ID"].count()*100/tot_st
# pass_rate
pass_ratecnt = len(student_data.loc[((student_data["reading_score"] > 69) & (student_data["math_score"] > 69)), "Student ID"])
# pass_ratecnt
# pass_rate = (pass_mathcnt*100/tot_st)+(pass_readcnt*100/tot_st)
# pass_rate = pass_rate/2
pass_rate = (avg_read+avg_math)/2
pass_rate
summary_student_data = pd.DataFrame({
                                     "Average Math Score": avg_math,
                                     "Average Reading Score": avg_read,
                                     "% Passing Math": pass_mathcnt*100/tot_st,
                                     "% Passing Reading": pass_readcnt*100/tot_st,
                                     "% Overall Passing Rate": pass_rate,
                                     "Rowid": 1
                                    },index=[0])
# summary_student_data
district_summary_data["Total Students"] = district_summary_data["Total Students"].map("{:,}".format)
# .apply('{:,}'.format)
district_summary_data["Total Budget"] = district_summary_data["Total Budget"].map('${:,.2f}'.format)
# district_summary_data
merged_db = pd.merge(district_summary_data,summary_student_data, on="Rowid", how="outer")
cleaned_merged_db = merged_db
del cleaned_merged_db['Rowid']
cleaned_merged_db

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
# school_data_complete_df.columns
# school_data_complete_df.describe()
# school_data_complete_df.head() 
renamed_school_data_df=school_data_complete_df.rename(columns={"type":"School Type","size":"Total Students","budget":"Total School Budget"})
# renamed_school_data_df.head()
cleaned_school_summary_df = renamed_school_data_df[["school_name","School Type","Total Students","Total School Budget","reading_score","math_score"]]
cleaned_school_summary_df.head()
#   Counting Math pass count by school....
math_data = cleaned_school_summary_df[["school_name","math_score"]].copy()
# math_data
math = math_data[math_data["math_score"] > 69]
# math
mathgrp2 = math.groupby(["school_name"])
# mathgrp2.head()
math_passcnt = mathgrp2.count()
math_passcnt
# Counting Reading pass count by school...
read_data = cleaned_school_summary_df[["school_name","reading_score"]].copy()
# read_data
read_pass = read_data[read_data["reading_score"] > 69]
# read_pass
readgrp = read_pass.groupby(["school_name"])
# readgrp.head()
read_passcnt = readgrp.count()
# read_passcnt.head()
pass_cnt = pd.merge(math_passcnt, read_passcnt, on="school_name")
pass_cnt = pass_cnt.rename(columns={"math_score":"Math Pass Count","reading_score":"Reading Pass Count"})
# pass_cnt.head()
school_summary_type = cleaned_school_summary_df[["school_name","School Type","Total School Budget"]].copy()
school_summary_type = school_summary_type.drop_duplicates()
# school_summary_type.head()
school_type = school_summary_type.set_index('school_name')
# school_type.head()
school_summary_group_df = cleaned_school_summary_df.groupby(["school_name"])
# school_summary_group_df.head()
school_summary_group_cnt = school_summary_group_df[["school_name"]].count()
# school_summary_group_cnt 
school_summary_group_cnt.columns = ['Total Students']
# school_summary_group_cnt.head()
school_summary_group_avg = school_summary_group_df[["math_score","reading_score"]].mean()
school_summary_group_avg = school_summary_group_avg.rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
# school_summary_group_avg.head()
merge1 = pd.merge(school_summary_group_cnt, school_type, on="school_name").merge(school_summary_group_avg, on="school_name")
# merge1.head()
####################   All columns required for calculations (merge3) used for other modules...
#
merge3 = pd.merge(merge1, pass_cnt, on="school_name")
merge3["Per Student Budget"] = (merge3["Total School Budget"]/merge3["Total Students"])
merge3["% Passing Math"] = (merge3["Math Pass Count"]/merge3["Total Students"])*100
merge3["% Passing Reading"] = (merge3["Reading Pass Count"]/merge3["Total Students"])*100
merge3["% Overall Passing Rate"] = (merge3["% Passing Math"]+merge3["% Passing Reading"])/2
# merge3.head()
# merge3.columns
#
#     School Summary Trimmed Data unformatted.....
#
# school_summary_data_full_df = merge3[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate', 'Math Pass Count','Reading Pass Count']]
school_summary_data_df = merge3[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
# school_summary_data_df.columns
# school_summary_data_df.head()
top_performing_schools_df = school_summary_data_df.sort_values(["% Overall Passing Rate"],ascending=False)
top_performing_schools_df["Total School Budget"] = top_performing_schools_df["Total School Budget"].map("${:.2f}".format)
top_performing_schools_df["Per Student Budget"] = top_performing_schools_df["Per Student Budget"].map("${:.2f}".format)
top_performing_schools_df.index.name=None
top_performing_schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,$1081356.00,$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,$1043130.00,$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,$585858.00,$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,$917500.00,$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,$1319574.00,$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
Bottom_performing_schools_df = school_summary_data_df.sort_values(["% Overall Passing Rate"])
Bottom_performing_schools_df["Total School Budget"] = Bottom_performing_schools_df["Total School Budget"].map("${:.2f}".format)
Bottom_performing_schools_df["Per Student Budget"] = Bottom_performing_schools_df["Per Student Budget"].map("${:.2f}".format)
Bottom_performing_schools_df.index.name=None
Bottom_performing_schools_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,$2547363.00,$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,$1884411.00,$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,$1910635.00,$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,$3094650.00,$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,$1763916.00,$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# school_data_complete_df.head()
# school_data_complete_df["grade"].unique()
#      Filtering required columns.... for this
stage1 = school_data_complete_df[["school_name","grade","math_score"]]
stage1.head()
stage2 = stage1.set_index("school_name")
stage2.head()
#      9th grade...
stage3 = stage2.loc[stage2["grade"] == "9th", :]
stage9 = stage3.loc[:, ["math_score"]]
stage9 = stage9.rename(columns={"math_score":"9th"})
stage9 = stage9.groupby(level=0).mean()
stage9.head()
#     10th grade...
stage4 = stage2.loc[stage2["grade"] == "10th", :]
stage10 = stage4.loc[:, ["math_score"]]
stage10 = stage10.rename(columns={"math_score":"10th"})
stage10 = stage10.groupby(level=0).mean()
#     11th grade...
stage5 = stage2.loc[stage2["grade"] == "11th", :]
stage11 = stage5.loc[:, ["math_score"]]
stage11 = stage11.rename(columns={"math_score":"11th"})
stage11 = stage11.groupby(level=0).mean()
#     12th grade...
stage6 = stage2.loc[stage2["grade"] == "12th", :]
stage12 = stage6.loc[:, ["math_score"]]
stage12 = stage12.rename(columns={"math_score":"12th"})
stage12 = stage12.groupby(level=0).mean()
stage12.head()
mathgrade_df = pd.concat([stage9, stage10, stage11, stage12], axis=1)
mathgrade_df.index.name=None
mathgrade_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
#      Filtering required columns.... for this
stage31 = school_data_complete_df[["school_name","grade","reading_score"]]
stage31.head()
stage32 = stage31.set_index("school_name")
# stage32.head()
#      9th grade...
stage33 = stage32.loc[stage2["grade"] == "9th", :]
stage39 = stage33.loc[:, ["reading_score"]]
stage39 = stage39.rename(columns={"reading_score":"9th"})
stage39 = stage39.groupby(level=0).mean()
#     10th grade...
stage34 = stage32.loc[stage2["grade"] == "10th", :]
stage310 = stage34.loc[:, ["reading_score"]]
stage310 = stage310.rename(columns={"reading_score":"10th"})
stage310 = stage310.groupby(level=0).mean()
#     11th grade...
stage35 = stage32.loc[stage2["grade"] == "11th", :]
stage311 = stage35.loc[:, ["reading_score"]]
stage311 = stage311.rename(columns={"reading_score":"11th"})
stage311 = stage311.groupby(level=0).mean()
#     12th grade...
stage36 = stage32.loc[stage2["grade"] == "12th", :]
stage312 = stage36.loc[:, ["reading_score"]]
stage312 = stage312.rename(columns={"reading_score":"12th"})
stage312 = stage312.groupby(level=0).mean()
stage312.head()
readgrade_df = pd.concat([stage39, stage310, stage311, stage312], axis=1)
readgrade_df.index.name=None
readgrade_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 616, 646, 676]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# school_data_complete_df.head()
#      Filtering required columns.... for this
sp_stage1 = school_data_complete_df[["size","budget","math_score","reading_score"]]
# sp_stage1.head()
sp_stage1 = sp_stage1.assign(student_budget=sp_stage1['budget']/sp_stage1['size'])
# sp_stage1.describe()
# sp_stage1.head()
sp_stage1['Spending Ranges (Per Student)'] = pd.cut(sp_stage1["student_budget"], spending_bins, labels=group_names)
# sp_stage1.head()
stage_grp = sp_stage1.groupby(["Spending Ranges (Per Student)"])
# stage_grp.head()
bin_cnt = stage_grp[["Spending Ranges (Per Student)"]].count()
bin_cnt.columns = ['bin_count']
# bin_cnt
# 
bin_avg = sp_stage1[["Spending Ranges (Per Student)","math_score","reading_score"]].copy()
avg_grp = bin_avg.groupby(["Spending Ranges (Per Student)"])
# avg_grp.head()
bin_score = avg_grp[["math_score","reading_score"]].mean()
bin_score.columns = ['Average Math Score','Average Reading Score']
# bin_score
bin_math = bin_avg[["Spending Ranges (Per Student)","math_score"]].copy()
bin_math = bin_math[bin_avg["math_score"] > 69]
bin_math = bin_math.groupby(["Spending Ranges (Per Student)"])
bin_math = bin_math.count()
# bin_math
bin_reading = bin_avg[["Spending Ranges (Per Student)","reading_score"]].copy()
bin_reading = bin_reading[bin_avg["reading_score"] > 69]
bin_reading = bin_reading.groupby(["Spending Ranges (Per Student)"])
bin_reading = bin_reading.count()
# bin_reading
bin_pass_score = pd.merge(bin_cnt, bin_math, on="Spending Ranges (Per Student)").merge(bin_reading, on="Spending Ranges (Per Student)")
bin_pass_score["% Passing Math"] = (bin_pass_score["math_score"]/bin_pass_score["bin_count"])*100
bin_pass_score["% Passing Reading"] = (bin_pass_score["reading_score"]/bin_pass_score["bin_count"])*100
# bin_pass_score
bin_pass_perc = bin_pass_score[["% Passing Math","% Passing Reading"]]
# bin_pass_perc
bin_merge1 = pd.merge(bin_cnt, bin_score, on="Spending Ranges (Per Student)").merge(bin_pass_perc, on="Spending Ranges (Per Student)")
bin_merge1["% Overall Passing Rate"] = (bin_merge1["% Passing Math"]+bin_merge1["% Passing Reading"])/2
# bin_merge1
scores_by_school_spending_df = bin_merge1[['Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
scores_by_school_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2001, 5001]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# school_data_complete_df.head()
#      Filtering required columns.... for this
sp_stage11 = school_data_complete_df[["size","math_score","reading_score"]]
# sp_stage11 = sp_stage11.rename(columns={"size":"School Size"})
sp_stage11.head()
# sp_stage1.describe()

,size,math_score,reading_score
0,2917,79,66
1,2917,61,94
2,2917,60,90
3,2917,58,67
4,2917,84,97


In [15]:
sp_stage11['School Size'] = pd.cut(sp_stage11["size"], size_bins, labels=group_names2)
sp_stage11.head()

/Users/rselvanayagam/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,size,math_score,reading_score,School Size
0,2917,79,66,Large (2000-5000)
1,2917,61,94,Large (2000-5000)
2,2917,60,90,Large (2000-5000)
3,2917,58,67,Large (2000-5000)
4,2917,84,97,Large (2000-5000)


In [16]:
stage_grp11 = sp_stage11.groupby(["School Size"])
# stage_grp11.head()
bin_cnt11 = stage_grp11[["School Size"]].count()
bin_cnt11.columns = ['bin_count']
bin_cnt11
# 
bin_avg11 = sp_stage11[["School Size","math_score","reading_score"]].copy()
avg_grp11 = bin_avg11.groupby(["School Size"])
# avg_grp11.head()
bin_score11 = avg_grp11[["math_score","reading_score"]].mean()
bin_score11.columns = ['Average Math Score','Average Reading Score']
bin_score11
bin_math11 = bin_avg11[["School Size","math_score"]].copy()
bin_math11 = bin_math11[bin_avg["math_score"] > 69]
bin_math11 = bin_math11.groupby(["School Size"])
bin_math11 = bin_math11.count()
# bin_math11
bin_reading11 = bin_avg11[["School Size","reading_score"]].copy()
bin_reading11 = bin_reading11[bin_avg["reading_score"] > 69]
bin_reading11 = bin_reading11.groupby(["School Size"])
bin_reading11 = bin_reading11.count()
# bin_reading11
bin_pass_score11 = pd.merge(bin_cnt11, bin_math11, on="School Size").merge(bin_reading11, on="School Size")
bin_pass_score11["% Passing Math"] = (bin_pass_score11["math_score"]/bin_pass_score11["bin_count"])*100
bin_pass_score11["% Passing Reading"] = (bin_pass_score11["reading_score"]/bin_pass_score11["bin_count"])*100
bin_pass_score11
bin_pass_perc11 = bin_pass_score11[["% Passing Math","% Passing Reading"]]
# bin_pass_perc11
bin_merge11 = pd.merge(bin_cnt11, bin_score11, on="School Size").merge(bin_pass_perc11, on="School Size")
bin_merge11["% Overall Passing Rate"] = (bin_merge11["% Passing Math"]+bin_merge11["% Passing Reading"])/2
# bin_merge11
scores_by_school_size_df = bin_merge11[['Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
scores_by_school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [17]:
# school_data_complete_df.head()
#      Filtering required columns.... for this
sp_stage22 = school_data_complete_df[["type","math_score","reading_score"]]
sp_stage22 = sp_stage22.rename(columns={"type":"School Type"})
# sp_stage22.head()
stage_grp22 = sp_stage22.groupby(["School Type"])
# stage_grp22.head()
bin_cnt22 = stage_grp22[["School Type"]].count()
bin_cnt22.columns = ['bin_count']
bin_cnt22
# 
bin_avg22 = sp_stage22[["School Type","math_score","reading_score"]].copy()
avg_grp22 = bin_avg22.groupby(["School Type"])
# avg_grp22.head()
bin_score22 = avg_grp22[["math_score","reading_score"]].mean()
bin_score22.columns = ['Average Math Score','Average Reading Score']
bin_score22
bin_math22 = bin_avg22[["School Type","math_score"]].copy()
bin_math22 = bin_math22[bin_avg["math_score"] > 69]
bin_math22 = bin_math22.groupby(["School Type"])
bin_math22 = bin_math22.count()
# bin_math22
bin_reading22 = bin_avg22[["School Type","reading_score"]].copy()
bin_reading22 = bin_reading22[bin_avg["reading_score"] > 69]
bin_reading22 = bin_reading22.groupby(["School Type"])
bin_reading22 = bin_reading22.count()
# bin_reading22
bin_pass_score22 = pd.merge(bin_cnt22, bin_math22, on="School Type").merge(bin_reading22, on="School Type")
bin_pass_score22["% Passing Math"] = (bin_pass_score22["math_score"]/bin_pass_score22["bin_count"])*100
bin_pass_score22["% Passing Reading"] = (bin_pass_score22["reading_score"]/bin_pass_score22["bin_count"])*100
bin_pass_score22
bin_pass_perc22 = bin_pass_score22[["% Passing Math","% Passing Reading"]]
# bin_pass_perc22
bin_merge22 = pd.merge(bin_cnt22, bin_score22, on="School Type").merge(bin_pass_perc22, on="School Type")
bin_merge22["% Overall Passing Rate"] = (bin_merge22["% Passing Math"]+bin_merge22["% Passing Reading"])/2
# bin_merge22
scores_by_school_size_df = bin_merge22[['Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
scores_by_school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
